# <a id='toc1_'></a>[Mobile Robotics Project Report](#toc0_)

---
---
Marcus Cemes, Pable Paller, Adrien Pannatier,  Carolina Rodrigues Fidalgo


<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    This notebook describes the different parts of our project for the course micro-452: Basics of mobile robotics.<br/>
    In the following cells we will describe how the modules work and give a structure to run the project.
</div>

**Table of contents**<a id='toc0_'></a>    
- [Mobile Robotics Project Report](#toc1_)    
  - [Prerequisites](#toc1_1_)    
    - [Useful imports](#toc1_1_1_)    
  - [Introduction](#toc1_2_)    
    - [Architecture](#toc1_2_1_)    
    - [Project specificities](#toc1_2_2_)    
      - [The Signal class](#toc1_2_2_1_)    
      - [The Context](#toc1_2_2_2_)    
      - [The State](#toc1_2_2_3_)    
        - [State patches](#toc1_2_2_3_1_)    
      - [The Web UI](#toc1_2_2_4_)    
  - [Vision](#toc1_3_)    
  - [Filtering](#toc1_4_)    
    - [Prediction](#toc1_4_1_)    
    - [Update](#toc1_4_2_)    
  - [Global navigation](#toc1_5_)    
    - [Path optimisation](#toc1_5_1_)    
  - [Big Brain](#toc1_6_)    
    - [Significant updates](#toc1_6_1_)    
    - [Christmas animation](#toc1_6_2_)    
  - [Motion control](#toc1_7_)    
  - [Local navigation](#toc1_8_)    
  - [Putting it all together](#toc1_9_)    
    - [Running from the terminal](#toc1_9_1_)    
    - [Running from Jupyter](#toc1_9_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Prerequisites](#toc0_)


<div class="alert alert-block alert-info">
    <b>Tip:</b> In the latest Python/Jupyter version, it should no longer be necsesary to add our application module to the path. If you experience any import errors, convert the following block to code and run it before anything else.
</div>

This project has been tested on **Python 3.10** and **JupyterLab 3.5**, allowing for features such as:

- Python's new `match` statement
- Top-level async/await
- Integrated event loop in the notebook

To make sure that your environment is supported, try running the following:

In [ ]:
from report.utils import run_checks

run_checks()

If you have any missing requirements, convert the following into a code block and run it (a [venv](https://docs.python.org/3/library/venv.html) is recommended).

Additional installation instructions can be found in the project's [README.md](README.md).


### <a id='toc1_1_1_'></a>[Useful imports](#toc0_)

We use these libraries and imports throughout the notebook, run this cell to make them available.

In [ ]:
from pprint import pprint

from asyncio import create_task, sleep
import numpy as np
import matplotlib.pyplot as plt

from app.config import *

from report.map import *
from report import report_functions

___
## <a id='toc1_2_'></a>[Introduction](#toc0_)

### <a id='toc1_2_1_'></a>[Architecture](#toc0_)

Our project is composed by 8 different modules presented in the following figure:

<div align="center" style="padding: 1rem 0;">
    <figure>
        <img src="assets/report/Project_scheme.jpg" style="width: 40rem;" />
        <figcaption style="font-style: italic;">Communication between the modules of the project.</figcaption>
    </figure>
</div>

All modules are independant and interact with one another sharing a shared [Context class] 

Some modules use [signals] to react to certain events and thus be independant from timers.

The main module is [Big brain] which processes information from other modules and give orders.

1. Classes
2. Single instaniation
3. Shared context
4. Signals vs. composition
5. Module class inheritence
6. Web UI + control server

But before going into detail about the modules. Let's focus on the specificities of our project:

### <a id='toc1_2_2_'></a>[Project specificities](#toc0_)

#### <a id='toc1_2_2_1_'></a>[The Signal class](#toc0_)

The `Signal` class ([types.py](./app/utils/types.py))  is a custom asyncronous syncronisation primitive. It wraps `asyncio`'s `Event` object, exposing two methods:

- `async wait(timeout: float)`
- `trigger()`

When triggered, it will allow all other tasks that are `await`ing the signal to continue execution.

In [ ]:
from app.utils.types import Signal

signal = Signal()

async def wait_for_notification(id):
    print(f"Task {id} is waiting...")
    await signal.wait()
    print(f"Task {id} has been notified!")

# Create two tasks that will wait for the trigger
for i in range(3):
    create_task(wait_for_notification(i))

await sleep(2)
signal.trigger()

We use signals extensively to asyncronously communicate between modules when certain events occur, without any dependence between our classes.

#### <a id='toc1_2_2_2_'></a>[The Context](#toc0_)

One of the key elements in our project architecture is the `Context` class ([context.py](./app/context.py)). It allows us to to have a single shared point of access to important data, such as the mutable application state or to the connected Thymio nodes. It avoids the use of any global variables, and also allows us to create a simple mock interface for this notebook for demonstration purposes.

Let's create one now that we will use for the rest of the report.

In [ ]:
from app.context import Context
from app.state import State

from report.mock_pool import MockPool

ctx = Context(node=None, node_top=None, pool=MockPool(), state=State())

# Set configuration attributes from constants
ctx.state.physical_size = PHYSICAL_SIZE_CM
ctx.state.subdivisions = SUBDIVISIONS

pprint(ctx)

It contains a few signals, that are used to decouple our modules from each other whilst allowing them to communicate events, the Thymio nodes a shared state object and a few other items.

| **Key**          | **Type**       | **Description**                                                            |
|------------------|----------------|----------------------------------------------------------------------------|
| **node**         | `Node`         | The primary Thymio node                                                    |
| **node_top**     | `Node \| None` | An optinal secondary Thymio node                                           |
| **state**        | `State`        | The singleton `State` instance                                             |
| **pool**         | `Pool`         | A ProcessPoolExecutor for offloading CPU-intensive work to another process |
| **scene_update** | `Signal`       | A signal to indicate that the map has been changed                         |
| **pose_update**  | `Signal`       | A signal to indicate that the robot has moved                              |
| **debug_update** | `boolean`      | Queue a single iteration of vision image processing debugging (GUI)        |

#### <a id='toc1_2_2_3_'></a>[The State](#toc0_)

The `State` class was originally created for a single purpose: simplify the update process to the Web UI. The `State` class internally has a `Signal` that is used to notify other tasks that the state has been modified, propagating the changes to all connected WebSocket clients over the control server.

**Rule of thumb**: anything that is to be viewed on the Web UI should belong in state. Anything else can be a private class attribute.

##### <a id='toc1_2_2_3_1_'></a>[State patches](#toc0_)

The control server creates a `ChangeListener` for each WebSocket connection, which listens for state changes and creates a minimum patch from the last call. In the following demo, we create three listeners, each with variying amounts of "busyness". Each task is able to independently track changes to reach a consistent version of the state.

In [ ]:
from report.introduction import scenario_state_patches

await scenario_state_patches()

> Understanding the implementation is not vital for this project. It allows for each WebSocket connection to have its own `ChangeListener`, with it's own dictionary of changes since the last call to `get_patch()`, ensuring that even if multiple clients are connected simultaneously, they can receive constient state updates. When a WebSocket connection is first made, the entire `State` is serialised and sent to the browser. This assited us in collaborative development and debugging.

#### <a id='toc1_2_2_4_'></a>[The Web UI](#toc0_)

To interact with our running application and to render a detailed map of detected obstacles, we developped a web application that connects to the Python application's control server on port 8080.

It's use is not mandatory for this report, but can be used to connect to the application when it's run in its entirety at the end of this report. For more information, see the [README.md](./README.md). It requires a recent version of [Node.js®](https://nodejs.org) to be installed on your system.

```powershell
$ cd ui
$ npm install
$ npm run build
$ npm run preview
```

<div align="center" style="padding: 1rem 0;">
    <figure>
        <img src="assets/report/ui.png" style="width: 60rem;" />
        <figcaption style="font-style: italic;">A screenshot of the Web UI with its various options</figcaption>
    </figure>
</div>

For the purposes of this report, we have developed a simplified view of the map that makes use of `matplotlib`. Throughout the report, we will make use of the plotting function to visualise the application state.

In [ ]:
plot_map(ctx, "Example map of the current state")

___
## <a id='toc1_3_'></a>[Vision](#toc0_)

In [ ]:
from app.vision import Vision

In [ ]:
vision = Vision(ctx, external=False, live=False, image_path="assets/test_frame_01.jpg")
reset(ctx)

In [ ]:
vision.calibrate();

In [ ]:
obs = vision.next()

In [ ]:
vision_pose = (obs.back[0], obs.back[1], report_functions.angle(obs.back, obs.front))
print(vision_pose)

display the vision retranscription

In [ ]:
ctx.state.position = (obs.back[0], obs.back[1])
ctx.state.orientation = report_functions.angle(obs.back, obs.front)
ctx.state.obstacles = obs.obstacles

In [ ]:
plot_map(ctx, "vision retranscription")

In [ ]:
show_picture("assets/test_frame_01.jpg")

___
## <a id='toc1_4_'></a>[Filtering](#toc0_)

Another important module of our project is filtering. Filtering is taking the information given by sensors and the camera to be able to guess where the thymio is on the map and where it's going.</br>
For this project we decieded to use an [Extanded Kalman Filter](app/EXF.py) and an intermediate module called [filtering](app/filtering.py) to comunicate with the main module.</br>
The EKF was chosen for its capability of predicting a non-linear evolution of the state and also to be able to dissociate information coming from the motors and information given by the camera.</br>
For this purpose, two functions where used: [predict](#toc1_4_1_) and [update](#toc1_4_2_). We will come back to these parts in the subsections below.
___


first let's import the module 

In [ ]:
from app.filtering import Filtering

In [ ]:
#run this cell to reset this section
#creation of the filter
filtering = Filtering(ctx)
reset(ctx)
report_functions.start_movement_simulation(filtering)

### <a id='toc1_4_1_'></a>[Prediction](#toc0_)

A specificity of our project is the time delay between camera and motor sensors. Since the motor sensors response is faster than the one from vision, what we do is predict the position of the robot with the motor sensors at a greater frequency than the update with vision information. The time interval between each prediction is dynamic and adapts to correctly predict each time the motor sensors send variables.

The next position and orientation are then predicted.

Let us run it to see how it works
___

Let's take a look at our current position.

In [ ]:
report_functions.print_pose(ctx)

In [ ]:
plot_map(ctx, "Filtering Map")

To execute a filtering step let's make the Thymio move forward:

In [ ]:
filtering.process_event({"motor.left.speed": [70], "motor.right.speed": [70]})
report_functions.stop_movement_simulation(filtering)

The state should now be updated

In [ ]:
report_functions.print_pose(ctx)
plot_map(ctx, "Filtering Map")

Let's do it again, turning the robot this time

In [ ]:
filtering.process_event({"motor.left.speed": [50], "motor.right.speed": [-50]})
report_functions.stop_movement_simulation(filtering)

In [ ]:
report_functions.print_pose(ctx)
plot_map(ctx, "Filtering Map")

we see that filtering is correctly predicting the progression of the simulated robot

### <a id='toc1_4_2_'></a>[Update](#toc0_)

To reduce the error caused by motors, vision must be used as much as possible. If vision is lost for a while, the update step brings back the Thymio to it's correct position taking into consideration the computed prediction since it's disapearing.
___

now to display the update function, we use the image and position of the robot given by the [Vision module](#toc1_3_)   

The position after our prediction step is:

In [ ]:
report_functions.print_pose(ctx)

now we update the position with the one given by vision

In [ ]:
vision_position = vision #get image from vision
filtering.update(vision_pose)

after the vision update, the new position is:

In [ ]:
report_functions.print_pose(ctx)
plot_map(ctx, "Filtering Map")

___
## <a id='toc1_5_'></a>[Global navigation](#toc0_)

In [ ]:
from app.global_navigation import GlobalNavigation

global_nav = GlobalNavigation(ctx)

_Source: [global_navigation.py](./app/global_navigation.py)_

At the moment our map is completely empty. In order to invoke our pathfinding algorithm, we need to set the Thymio's position (<b style="color: red;">red</b>) and a desired end point (<b style="color: cyan;">cyan</b>).

In [ ]:
# Start with a fresh new map
reset(ctx)

# Update the state
ctx.state.position = (10, 10)
ctx.state.end = (90, 110)

# Recompute the path
await global_nav._recompute_path()

# Plot the result
plot_map(ctx, "Path-finding map")

**🎉 We have a path!**

Let's not celebrate yet, we can make this much more interesting. Our pathfinding uses an implementation of the [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm). While this may not be the most fastest or most efficient algorithm, it _does_ guarantee optimal results for a given graph.

Our node graph is a grid of equaly spaced cells, where edges are made between directly adjacent cells and diagonal cells, with a cost of 1 and $\sqrt{2}$, respectively. This allows our robot to also travel diagonally and avoid grid-like movement.

<div align="center" style="padding: 1rem 0;">
    <figure>
        <img src="assets/report/dijkstra-nodes.png" style="width: 8rem;" />
        <figcaption style="font-style: italic;">Edges for a graph node to its adjacent nodes.</figcaption>
    </figure>
</div>

Ideally, we would like all nodes to be connected to every other node, given that there is a line-of-sight between them, however this comes with a huge computational burden. This simple implementation (mixed with our path optimisation algorithm covered later) provides adequate results in roughly 50 ms for 64 subdivisions.

Time for some obstacles! Obstacles come directly from the vision module as a matrix of 8-bit integers, where any value other an 0 represent an occupied cell. The size of the matrix coincides with the node graph, an occupied cell will mark a node as unvisitable.

**Note:** While we could have also used a matrix of booleans, integers are slightly easier to work with and compose with other functions such as convolutions. Additionally, it has a smaller JSON serialisation footprint for sending over WebSockets to the Web UI.

In [ ]:
# Reset the matrix
ctx.state.obstacles[:,:] = 0

# Add two obstacles
ctx.state.obstacles[30:45, 40:55] = 1
ctx.state.obstacles[20:30, 5: 20] = 1

# Recompute the path
await global_nav._recompute_path()

# Plot the result
plot_map(ctx, "Path-finding map")

The dark-blue region represents the obstacles that we set. The cyan regions are the obstacle boundaries that are generated using a convolution of a circular kernel matrix.

In [ ]:
kernel = global_nav._safety_margin_kernel()

plot_image(kernel, "Obstacle boundary convolution kernel", colourbar=True)

The boundary is equaly treated as unvisitable by the pathfinding algorith. This ensures that the Thymio does not hit any objects with its wheels as it's navigating the path. It a gap is too small, the Thymio will not try to fit through it.

In [ ]:
# Reset the matrix
ctx.state.obstacles[:,:] = 0

# Add two obstacles with a gap
ctx.state.obstacles[31:33, 15:29] = 1
ctx.state.obstacles[31:33, 42:50] = 1

# Recompute the path
await global_nav._recompute_path()

# Plot the result
plot_map(ctx, "Path-finding map")

### <a id='toc1_5_1_'></a>[Path optimisation](#toc0_)

Our chosen node graph constraints our robots movements more or less to a grid, with the ability of also being able to move diagonally. This creates a lot of path waypoints, but also creates less natural-like movement.

To combat this, we have an additional post-pathfinding step to iteratively try to reduce the path to as few waypoints as necessary that have free line of sight between them. This does not find the most optimal solution, but the results are more than adequate.

The algorithm goes as follows:

1. For $i = 2,...,M-1$, where $M$ is the number of waypoints
2. Check whether there is a free line-of-sight between waypoints $w_{i-1}$ and $w_{i+1}$
3. If yes, remove the waypoint $w_i$, otherwise consider the next waypoint $w_{i+1}$

In order to calculate free line-of-sight, it's necessary to enumerate all map cells that the segment between $w_{i-1}$ and $w_{i+1}$ travels through to check their occupation status. This is done using an algorithm such as [Bresenham's line algorithm](https://en.wikipedia.org/wiki/Bresenham%27s_line_algorithm). We found an improved version that also uses integer-only math [here](https://playtechs.blogspot.com/2007/03/raytracing-on-grid.html).

In [ ]:
from app.path_finding.path_optimiser import raytrace

p1 = (1, 2)
p2 = (8, 6)

matrix = np.zeros((10, 10))

for (i, j) in raytrace(p1, p2):
    matrix[j][i] = 1

plot_raytrace(matrix, p1, p2)

Let's put this into practice using the path that we found previously.

In [ ]:
from app.path_finding.path_optimiser import PathOptimiser

map = create_map(ctx)

optimiser = PathOptimiser(map)

# The output of the pathfinding algorithm is in physical space [cm],
# it needs to be converted back into map indicies for this demonstration.
# Usually the path optimisation is done as an intermediate step by the
# path finding algorithm, before the conversion to physical space.
path = path_to_coords(ctx.state.path)

# The optimiser mutates the input list directly
optimised_path = optimiser.optimise(path.copy())

plot_path_optimisation(path, optimised_path, map)

Path optimisation can be toggled dynamically by setting the `ctx.state.optimise` boolean flag.

In [ ]:
# Enable path optimisation
ctx.state.optimise = True

# Reset the matrix
ctx.state.obstacles[:,:] = 0

# Add two obstacles
ctx.state.obstacles[30:45, 40:55] = 1
ctx.state.obstacles[20:30, 5: 20] = 1

await global_nav._recompute_path()

plot_map(ctx, "Path-finding map")

___
## <a id='toc1_6_'></a>[Big Brain](#toc0_)

In [ ]:
import app.big_brain

big_brain = app.big_brain.BigBrain(ctx)

_Source: [big_brain.py](app/big_brain.py)_

The **Big Brain** module is in charge of the main application logic. It's purpose is to initialise the other modules and to orchestrate the flow of data between them.

The submodules are able to run autonomously in the background of the application using Python's `with` statement and the `asyncio` library. The `__enter__` and `__exit__` methods are used to initialise and clean up the resources used by the submodules, such as long running tasks or registering and unregistering Thymio event handlers.

Once the submodules are initialised, the main loop of the application is started. This loop is responsible for acquiring new data from the vision module and providing this data to the filtering and global navigation modules, which then trickle down to motion control and to the Web UI.

### <a id='toc1_6_1_'></a>[Significant updates](#toc0_)

To reduce the number of map updates from the vision module, we have implemented a simple thresholding function that checks whether the $L_1$ norm between the current and previous map is greater than a given threshold (i.e. a certain number of grid squares have changed). This reduces redundant pathfinding computation and also renders motion control more stable, as it is less likely to be interrupted by a sudden change in the map and path.

In [ ]:
ctx.state.obstacles = np.zeros((SUBDIVISIONS, SUBDIVISIONS), dtype=np.int8)
new_map = np.zeros((SUBDIVISIONS, SUBDIVISIONS), dtype=np.int8)

# Changing a signal square does not report a significant change
new_map[1, 1] = 1

big_brain.significant_change(new_map)

In [ ]:
# Changing more than the threshold of squares should report a significant change
new_map[1:5, 1:5] = 1

big_brain.significant_change(new_map, threshold=10)

If the new map is deemed to have a significant change, the obstacle map is updated in `ctx.state` and the `ctx.scene_update` signal is triggered, which in turn triggers the global navigation module to recompute the path.

Regardless of whether the map has changed or not, the new detected position is sent to the filtering module in order to update the robot's position and attempt to correct any drift.

### <a id='toc1_6_2_'></a>[Christmas animation](#toc0_)

Once the robot reaches the arrival point, it will start to play a celebration sequence. This is also handled by the big brain module, invoking the `ChristmasCelebration` class to play various actions, such as dropping the bauble using the second Thymio node.

___
## <a id='toc1_7_'></a>[Motion control](#toc0_)

The block motion control has the responsability to move the thymio. It has 2 types of control, positional control 'controlPosition()' which has the objective to go to a special set of coordinates. And reactive control 'controlWithDistance()' which has the objective to avoid any walls and get around them following their left.

Motion control also has the responsability to update the targeted waypoint if it is reached and commmunicate to the rest of the program if the last waypoint is reached. 

1) Update the waypoint when arrived
2) If arrived to the end of the path, change the state to let the rest of the program know 

In [ ]:
from app.motion_control import MotionControl
#creation of the filter
control = MotionControl(ctx)

Let's simulate a path following !

SETUP :
1) We have do define a path (noramlly given by BigBrain in Global-nav)
2) We have to define which waypoint we are aiming


SIMULATION : 
1) We control the Thymio to go to the specified waypoint we want 
2) We update his position by updating filtering

In [ ]:
## SETUP
#1)
ctx.state.boundary_map = None
ctx.state.obstacles = None
report.map.reset(ctx)

ctx.state.path = [[10, 0], [10, 10], [20, 10], [0,0], [0,0]] #the path we want to follow
ctx.state.position = [0,0]
ctx.state.orientation = 0
filtering.ekf.__init__(ctx.state.position, ctx.state.orientation) #let's force the Thymio position to [0,0] and an orientation of 0
ctx.state.next_waypoint_index = 0 # we want to have the first waypoint 
ctx.state.reactive_control = False


#2)
control.setNewWaypoint(0) #load the next waypoint we want to go to
print("objective waypoint : ",control.waypoint) 

In [ ]:
import time
from IPython.display import clear_output

while(ctx.state.next_waypoint_index < len(ctx.state.path)-1):
    clear_output(wait=True)
    report.map.plot_map(ctx, "Path-finding map")
    
    (arrived, vLC, vRC) = control.controlPosition() #1)Get control
    if(arrived):
        control.setNewWaypoint(1) #load the next waypoint to aim if arrived to the previous one
    
    filtering.process_event({"motor.left.speed": [int(vLC)], "motor.right.speed": [int(vRC)]}) #    2)force an update of position 
    time.sleep(0.6)

We need to introduce the Local Navigation block to understand completely the 'controlWithDistance()' function. The full explaination can be found in the [local navigation section](#toc1_8_).

Motion control receives the distance array which contains how far from the obstacle the Thymio is.

Distance control is very simple:

1) If no obstacles : Thymio will try to go forward and turn a bit to the right trying to find wall to follow on his leften side
2) If an obstacle is close enough: 

    2.1) If the Thymio is really close to the obstalce, it will move backward
    
    2.2) It will move to the right trying to avoid the obstacle

Depending if the triggered sensor is frontal or from the sides, the reaction will be similar but more or less accentuated.


In [ ]:
from app.local_navigation import LocalNavigation
localNav = LocalNavigation(ctx, MotionControl)

localNav.process_event(variables={"prox.horizontal": [2800,3000,3300,3000,3400,2000,2000]}) #simulate the sensors 
distArray = localNav.getDistanceArray() #the distance array to define the real distance between the sensor and the obstacle
ctx.state.relative_distances = distArray
print(distArray[:-2])
(arrived, vLC, vRC) = control.controlWithDistance()
print(vLC, vRC)
print("The Thymio is going foward and turning a bit to the right to try figure out where the wall is ")

localNav.process_event(variables={"prox.horizontal": [2800,3000,4500,3000,3400,2000,2000]}) #simulate the sensors 
distArray = localNav.getDistanceArray() #the distance array to define the real distance between the sensor and the obstacle
ctx.state.relative_distances = distArray

print(distArray[:-2])
(arrived, vLC, vRC) = control.controlWithDistance()
print(vLC, vRC)
print("The Thymio is going backward and turning to the left to avoid the wall")

In our program, we are calling the funtion 'update_motor_control()' this function chooses the correct control defined in the Local navigation block. The function will also update the targeted waypoint if the thymio arrives to the waypoint

___
## <a id='toc1_8_'></a>[Local navigation](#toc0_)

The Local navigation block has the responsability to trigger the control by waypoint or the control with distance state depending if the Thymio senses an obstacle close to his sensors. It communicates the type of control through the state variable : 'reactive_control'. 

This block can also detect and locate obstacle positions with the function: 'getWallRelative()'

In [ ]:
from app.local_navigation import LocalNavigation

localNav = LocalNavigation(ctx, MotionControl)

This function does not exist in the project but for the report it is very useful because we can simulate a measure of the Thymio's sensors and update the State.

In [ ]:
def simulateSensors(sensorsValues):
    localNav.process_event(variables={"prox.horizontal": sensorsValues}) #simulate the sensors 

    distArray = localNav.getDistanceArray() #the distance array to define the distance [in cm] between the sensor and the obstacle
    ctx.state.relative_distances = distArray 

In [ ]:
simulateSensors([4000,3000,3300,4000,3400,2000,2000])
print(ctx.state.relative_distances)

The function 'getWallRelative()' returns the relative position of the sensed walls (relative because it depends on the position and orientation of the Thymio). This function could be useful if in the future we want to implement a scan of obstacles with the Thymio.

In [ ]:
simulateSensors([3000,3000,3300,4000,3400,2000,2000])
relative_ostacle_position = localNav.getWallRelative()
print(relative_ostacle_position)

Now we enter the main part of the block: the type of control trigger depending on the sensors Values.
The function 'should_freestyle()' defines if the Thymio should enter the reactive control mode and updates the state. It will be read by the Motion control block for it to apply the correct control. 


In [ ]:
ctx.state.reactive_control = False

simulateSensors([3000,3000,3000,3000,3400,2000,2000]) #Reading of sensors far away from an obstacle
print("relative distances : ",ctx.state.relative_distances[:-2])
localNav.should_freestyle()
print("should enter local navigation ? : ",ctx.state.reactive_control)


simulateSensors([3000,4500,4500,3000,3400,2000,2000])#Reading of sensors close to an obstacle, the front one at 4500
print("relative distances : ",ctx.state.relative_distances[:-2])
localNav.should_freestyle()
print("should enter local navigation ? : ",ctx.state.reactive_control)



___
## <a id='toc1_9_'></a>[Putting it all together](#toc0_)

Now that we have covered all the different modules of our application, we can put them together to run the whole thing.

There are two ways to run the application.

### <a id='toc1_9_1_'></a>[Running from the terminal](#toc0_)

This is the **recommended** approach, it's more robust and allows you to see better application logs, stop the application with Ctrl+C and also use keyboard input using stdin.


```powershell
$ python -m app
```


### <a id='toc1_9_2_'></a>[Running from Jupyter](#toc0_)

On modern versions of IPython (7+), it's possible to run the application from a Jupyter notebook using the top-level await syntax. From our experiments, the coloured terminal logging, GUI calibration window and keyboard input should all work.

Note that running the following cell will block the notebook kernel until the cell is interrupted (the future is cancelled), the kernal is restarted or the Web UI is used to terminal the application.

In [ ]:
from asyncio import CancelledError
from app.__main__ import init


try:
    await init()

except CancelledError:
    pass